# Task 2: Exploratory Data Analysis
---

 Background information

The BCG project team thinks that building a churn model to understand whether price sensitivity is the largest driver of churn has potential. The client has sent over some data and the AD wants you to perform some exploratory data analysis.

The data that was sent over includes:

    Historical customer data: Customer data such as usage, sign up date, forecasted usage etc
    Historical pricing data: variable and fixed pricing data etc
    Churn indicator: whether each customer has churned or not

Task

Sub-Task 1:

Perform some exploratory data analysis. Look into the data types, data statistics, specific parameters, and variable distributions. This first subtask is for you to gain a holistic understanding of the dataset. You should spend around 1 hour on this.

Sub-Task 2:

Verify the hypothesis of price sensitivity being to some extent correlated with churn. It is up to you to define price sensitivity and calculate it. You should spend around 30 minutes on this.

Sub-Task 3:

Prepare a half-page summary or slide of key findings and add some suggestions for data augmentation – which other sources of data should the client provide you with and which open source datasets might be useful? You should spend 10-15 minutes on this.


## Sub-Task 1
---

### Read Data
---

In [21]:
import logging as log
log.basicConfig(filename='./eda.log', level=log.DEBUG, format = '%(asctime)s %(levelname)s %(name)s %(message)s')


In [23]:
import pandas as pd

def read_file(file_name):
    file_extension = file_name.split(".")[-1].lower()

    if file_extension == "csv":
        return pd.read_csv(file_name)
    elif file_extension == "xlsx":
        return pd.read_excel(file_name)
    elif file_extension == "json":
        return pd.read_json(file_name)
    elif file_extension == "parquet":
        return pd.read_parquet(file_name)
    else:
        raise ValueError("Unsupported file format.")

In [22]:
DATA_ROOT_DIR = '../data/'

In [24]:
try:
    client_df = read_file(f'{DATA_ROOT_DIR}client_data.csv')
    log.debug(f'\nSuccessfully read {DATA_ROOT_DIR}client_data.csv')
except Exception as e:
    log.error(e)
    

In [26]:
#lets observe the data briefly
client_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14606 entries, 0 to 14605
Data columns (total 26 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              14606 non-null  object 
 1   channel_sales                   14606 non-null  object 
 2   cons_12m                        14606 non-null  int64  
 3   cons_gas_12m                    14606 non-null  int64  
 4   cons_last_month                 14606 non-null  int64  
 5   date_activ                      14606 non-null  object 
 6   date_end                        14606 non-null  object 
 7   date_modif_prod                 14606 non-null  object 
 8   date_renewal                    14606 non-null  object 
 9   forecast_cons_12m               14606 non-null  float64
 10  forecast_cons_year              14606 non-null  int64  
 11  forecast_discount_energy        14606 non-null  float64
 12  forecast_meter_rent_12m         

In [27]:
#Convert datetime objects to datetime data types.
for c in ['date_activ','date_end','date_modif_prod','date_renewal']:
    client_df[c] = pd.to_datetime(client_df[c])

In [29]:
# Add some new features.
client_df['contract_start_year'] = client_df['date_activ'].dt.year
client_df['contract_end_year'] = client_df['date_end'].dt.year

In [30]:
#find duplicates and missing data. 
def duplicate_and_missing(dataset, dataset_name):
    num_rows, num_columns = dataset.shape

    print(f"There are {num_rows} rows and {num_columns} columns in the dataset '{dataset_name}'")
    print("--" * 40)

    # Display missing values
    missing_values = dataset.isna().sum()
    if missing_values.sum() != 0:
        missing_values = missing_values[missing_values != 0]
        missing_proportion = missing_values / len(dataset)
        print(f"There are {missing_values.sum()} missing values")
        missing_df = pd.DataFrame({'count': missing_values, 'proportion': missing_proportion})
        print(missing_df)
        print("--" * 40)
    else:
        print("There are no missing values")

In [31]:
duplicate_and_missing(dataset=client_df, dataset_name='Client')

There are 14606 rows and 28 columns in the dataset 'Client'
--------------------------------------------------------------------------------
There are no missing values


In [32]:
#Lets find all categorical data.
def describe_categorical(dataset):
    cat_columns = dataset.select_dtypes(include=['object']).columns.tolist()
    if len(cat_columns) != 0:
        print("Categorical variables are:", cat_columns)
        print("==" * 40)
        for cat in cat_columns:
            describe_frame = dataset[cat].value_counts().reset_index(name='count')
            describe_frame['proportion'] = describe_frame['count'] / len(dataset)
            print(describe_frame)
            print("--" * 40)
    else:
        print("There are no categorical variables in the dataset")

In [33]:
describe_categorical(dataset=client_df)

Categorical variables are: ['id', 'channel_sales', 'has_gas', 'origin_up']
                                     id  count  proportion
0      24011ae4ebbe3035111d65fa7c15bc57      1    0.000068
1      2fae70276cd7a4874a2aefcd68d5a184      1    0.000068
2      a7a9dac0ffc2ad56c66a1b08d53d0e51      1    0.000068
3      6cfbce2099a4163ea4fe422fb28829ea      1    0.000068
4      4289e929ecd35a0754e697feb24a091b      1    0.000068
...                                 ...    ...         ...
14601  0b69432a3ceee7763bf86ffe30c6bc2d      1    0.000068
14602  6cabca88c760284183f51f84e4a27b1e      1    0.000068
14603  de92494d7140dc94be6a1ebafaf98056      1    0.000068
14604  a8e828bfad0869527e9acdd39a36cc91      1    0.000068
14605  563dde550fd624d7352f3de77c0cdfcd      1    0.000068

[14606 rows x 3 columns]
--------------------------------------------------------------------------------
                      channel_sales  count  proportion
0  foosdfpfkusacimwkcsosbicdxkicaua   6754    0.462413


In [37]:
#I transformed the "channel_sales" and "origin_up" values into numerical data types using label encoding, as they were initially unclear.
from sklearn.preprocessing import LabelEncoder
# label encoding
channel_encoder = LabelEncoder()
client_df['channel_sales'] = channel_encoder.fit_transform(client_df['channel_sales'])
origin_encoder = LabelEncoder()
client_df['origin_up'] = origin_encoder.fit_transform(client_df['origin_up'])

In [38]:
#Nmeric Data

def describe_numeric(dataset):
    # Get numeric columns with discrete values and display their frequencies
    int_columns = dataset.select_dtypes(include='int').columns.tolist()
    if int_columns:
        print("Numeric variables with discrete values:", int_columns)
        print('=' * 80)
        for column in int_columns:
            value_counts = dataset[column].value_counts()
            proportions = value_counts / len(dataset)
            describe_frame = pd.DataFrame({'value': value_counts.index, 'count': value_counts, 'proportion': proportions})
            print(describe_frame)
            print('--' * 40)
    
    # Get numeric columns with continuous values and display their summary statistics
    float_columns = dataset.select_dtypes(include='float').columns.tolist()
    if float_columns:
        print("Numeric variables with continuous values:", float_columns)
        print('=' * 80)
        numeric_summary = dataset[int_columns + float_columns].describe().loc[['min', 'max', 'mean', '50%']].T.rename(columns={'50%': 'median'})
        print(numeric_summary)
        print('--' * 40)
    
    # Handle the case when there are no numeric variables in the dataset
    if not int_columns and not float_columns:
        print("There are no numeric variables in the dataset")


In [39]:
describe_numeric(dataset=client_df)

Numeric variables with discrete values: ['channel_sales', 'cons_12m', 'cons_gas_12m', 'cons_last_month', 'forecast_cons_year', 'nb_prod_act', 'num_years_antig', 'origin_up', 'churn', 'contract_start_year', 'contract_end_year']
               value  count  proportion
channel_sales                          
4                  4   6754    0.462413
0                  0   3725    0.255032
5                  5   1843    0.126181
7                  7   1375    0.094139
2                  2    893    0.061139
6                  6     11    0.000753
1                  1      3    0.000205
3                  3      2    0.000137
--------------------------------------------------------------------------------
            value  count  proportion
cons_12m                            
0               0    117    0.008010
2882597   2882597     27    0.001849
3329244   3329244     24    0.001643
1743025   1743025     18    0.001232
3926060   3926060     18    0.001232
...           ...    ...         

<format size=12> No Issues on numeric data

In [41]:
#Checking datetime data.

client_df[client_df.dtypes[client_df.dtypes=='datetime64[ns]'].index.tolist()].describe()

,date_activ,date_end,date_modif_prod,date_renewal
count,14606,14606,14606,14606
mean,2011-01-28 07:54:18.879912448,2016-07-27 20:48:26.422018560,2013-01-02 12:29:10.951663872,2015-07-21 06:59:00.353279488
min,2003-05-09 00:00:00,2016-01-28 00:00:00,2003-05-09 00:00:00,2013-06-26 00:00:00
25%,2010-01-15 00:00:00,2016-04-27 06:00:00,2010-08-12 00:00:00,2015-04-17 00:00:00
50%,2011-03-04 00:00:00,2016-08-01 00:00:00,2013-06-19 00:00:00,2015-07-27 00:00:00
75%,2012-04-19 00:00:00,2016-10-31 00:00:00,2015-06-16 00:00:00,2015-10-29 00:00:00
max,2014-09-01 00:00:00,2017-06-13 00:00:00,2016-01-29 00:00:00,2016-01-28 00:00:00


<format size=12> No issues on datetime objects.

In [45]:
#Analyze Price Data
try:
    price_df = read_file(f'{DATA_ROOT_DIR}price_data.csv')
    log.debug(f'\nSuccessfully read {DATA_ROOT_DIR}price_data.csv')
except Exception as e:
    log.error('\n'+ e)

In [46]:
price_df.head(3)

,id,price_date,price_off_peak_var,price_peak_var,price_mid_peak_var,price_off_peak_fix,price_peak_fix,price_mid_peak_fix
0,038af19179925da21a25619c5a24b745,2015-01-01,0.151367,0.0,0.0,44.266931,0.0,0.0
1,038af19179925da21a25619c5a24b745,2015-02-01,0.151367,0.0,0.0,44.266931,0.0,0.0
2,038af19179925da21a25619c5a24b745,2015-03-01,0.151367,0.0,0.0,44.266931,0.0,0.0


In [47]:
price_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193002 entries, 0 to 193001
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  193002 non-null  object 
 1   price_date          193002 non-null  object 
 2   price_off_peak_var  193002 non-null  float64
 3   price_peak_var      193002 non-null  float64
 4   price_mid_peak_var  193002 non-null  float64
 5   price_off_peak_fix  193002 non-null  float64
 6   price_peak_fix      193002 non-null  float64
 7   price_mid_peak_fix  193002 non-null  float64
dtypes: float64(6), object(2)
memory usage: 11.8+ MB


In [49]:
price_df['price_date'] = pd.to_datetime(price_df['price_date'])

In [50]:
duplicate_and_missing(dataset=price_df, dataset_name='Price')

There are 193002 rows and 8 columns in the dataset 'Price'
--------------------------------------------------------------------------------
There are no missing values


In [51]:
describe_categorical(dataset=price_df)

Categorical variables are: ['id']
                                     id  count  proportion
0      038af19179925da21a25619c5a24b745     12    0.000062
1      97589d95ec46ab31ef238d18a616403f     12    0.000062
2      97adbcd721fae4874f7ee287ea84200d     12    0.000062
3      d74b4544363be674855cf931b6b8af25     12    0.000062
4      27276020e48d73f589788d4e37c51879     12    0.000062
...                                 ...    ...         ...
16091  83cf18b07114e495ae8b7fb235e45ee2      8    0.000041
16092  3e459d61dc831e29f8a9a9a59f95efd2      8    0.000041
16093  c5dcd5c506e565aaabffa29bc1ec0a37      7    0.000036
16094  bf89f2d8c1b133a134fd93603cb4c947      7    0.000036
16095  15b36e47cf04bf151e3f4438d12672e5      7    0.000036

[16096 rows x 3 columns]
--------------------------------------------------------------------------------


In [52]:
describe_numeric(dataset=price_df)

Numeric variables with continuous values: ['price_off_peak_var', 'price_peak_var', 'price_mid_peak_var', 'price_off_peak_fix', 'price_peak_fix', 'price_mid_peak_fix']
                    min        max       mean     median
price_off_peak_var  0.0   0.280700   0.141027   0.146033
price_peak_var      0.0   0.229788   0.054630   0.085483
price_mid_peak_var  0.0   0.114102   0.030496   0.000000
price_off_peak_fix  0.0  59.444710  43.334477  44.266930
price_peak_fix      0.0  36.490692  10.622875   0.000000
price_mid_peak_fix  0.0  17.458221   6.409984   0.000000
--------------------------------------------------------------------------------


In [53]:
price_df['price_date'].describe()

count                           193002
mean     2015-06-16 12:50:49.933161216
min                2015-01-01 00:00:00
25%                2015-04-01 00:00:00
50%                2015-07-01 00:00:00
75%                2015-10-01 00:00:00
max                2015-12-01 00:00:00
Name: price_date, dtype: object

### Data Visualization
---

In [56]:
plot_bar_chart_with_percent_label(df=client_df, target_variable='churn', gap_label_bar=0.02)

KeyError: 'index'